# Use Spark and batch deployment to predict customer churn with `ibm-watson-machine-learning`

This notebook contains steps and code to develop a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.6 and Apache® Spark 2.4.

You will use a data set, **Telco Customer Churn**, which details anonymous customer data from a telecommunication company. Use the details of this data set to predict customer churn which is very critical to business as it's easier to retain existing customers rather than acquiring new ones.

## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Explore and visualize prediction results using the plotly package.
-  Deploy a model for batch scoring using Wastson Machine Learning API.


## Contents

This notebook contains the following parts:

1.	[Set up the environment](#setup)
2.	[Load and explore data](#load)
3.	[Create spark ml model](#model)
4.	[Persist model](#persistence)
5.	[Predict locally and visualize](#visualization)
6.	[Deploy and score in a Cloud](#scoring)
7.  [Clean up](#cleanup)
8.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>).

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [1]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [2]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [3]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First, create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [4]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [5]:
client.set.default_space(space_id)

'SUCCESS'

### Test Spark

In [6]:
try:
    from pyspark.sql import SparkSession
except:
    print('Error: Spark runtime is missing. If you are using Watson Studio change the notebook runtime to Spark.')
    raise

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration. 

In [7]:
import os
from wget import download

sample_dir = 'spark_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename = os.path.join(sample_dir, 'WA_FnUseC_TelcoCustomerChurn.csv')
if not os.path.isfile(filename):
    filename = download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/data/customer_churn/WA_FnUseC_TelcoCustomerChurn.csv", out=sample_dir)

In [8]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .option('nanValue', ' ')\
  .option('nullValue', ' ')\
  .load(filename)

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  count all records
-  show distribution of label classes

In [9]:
df_data.printSchema()

print("Number of fields: %3g" % len(df_data.schema))

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)

Number of fields:  21


As you can see, the data contains 21 fields. "Churn" field is the one we would like to predict (label).

In [10]:
print("Total number of records: " + str(df_data.count()))

Total number of records: 7043


Data set contains 7043 records.

Now you will check if all records have complete data.

In [11]:
df_complete = df_data.dropna()

print("Number of records with complete data: %3g" % df_complete.count())

Number of records with complete data: 7032


You can see that there are some missing values you can investigate that all missing values are present in `TotalCharges` feature. We will use dataset with missing values removed for model training and evaluation.


Now you will inspect distribution of classes in label column.

In [12]:
df_complete.groupBy('Churn').count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|   No| 5163|
|  Yes| 1869|
+-----+-----+



<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [13]:
(train_data, test_data, predict_data) = df_complete.randomSplit([0.8, 0.18, 0.02], 24)

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))
print("Number of records for prediction: " + str(predict_data.count()))

Number of records for training: 5621
Number of records for evaluation: 1263
Number of records for prediction: 148


As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [14]:
from pyspark.ml.feature import StringIndexer, IndexToString, RFormula
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the predictors to features vector and label feature convert to numeric.

In [15]:
lab = StringIndexer(inputCol = 'Churn', outputCol = 'label')
features = RFormula(formula = "~ gender + SeniorCitizen +  Partner + Dependents + tenure + PhoneService + MultipleLines + InternetService + OnlineSecurity + OnlineBackup + DeviceProtection + TechSupport + StreamingTV + StreamingMovies + Contract + PaperlessBilling + PaymentMethod + MonthlyCharges + TotalCharges - 1")

Next, define estimators you want to use for classification. Logistic Regression is used in the following example.

In [16]:
lr = LogisticRegression(maxIter = 10)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [17]:
pipeline_lr = Pipeline(stages = [features, lab , lr])

Now, you can train your Logistic Regression model using the previously defined **pipeline** and **train data**."

In [18]:
model_lr = pipeline_lr.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [19]:
predictions = model_lr.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Test dataset:")
print("Accuracy = %3.2f" % accuracy)

Test dataset:
Accuracy = 0.80


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist model


In this section you will learn how to store your pipeline and model in Watson Machine Learning repository using Python client libraries.

**Note**: Apache® Spark 2.4 is required.

#### Save training data in your Cloud Object Storage

ibm-cos-sdk library allows Python developers to manage Cloud Object Storage (COS).

In [20]:
import ibm_boto3
from ibm_botocore.client import Config

**Action**: Put credentials from Object Storage Service in Bluemix here.

In [21]:
cos_credentials = {
                  "apikey": "***",
                  "cos_hmac_keys": {
                    "access_key_id": "***",
                    "secret_access_key": "***"
                  },
                  "endpoints": "***",
                  "iam_apikey_description": "***",
                  "iam_apikey_name": "***",
                  "iam_role_crn": "***",
                  "iam_serviceid_crn": "***",
                  "resource_instance_id": "***"
                }

In [22]:
connection_apikey = cos_credentials['apikey']
connection_resource_instance_id = cos_credentials["resource_instance_id"]
connection_access_key_id = cos_credentials['cos_hmac_keys']['access_key_id']
connection_secret_access_key = cos_credentials['cos_hmac_keys']['secret_access_key']

**Action**: Define the service endpoint we will use. <br>
**Tip**: You can find this information in Endpoints section of your Cloud Object Storage instance dashbord.

In [23]:
service_endpoint = 'https://s3.us.cloud-object-storage.appdomain.cloud'

You also need IBM Cloud authorization endpoint to be able to create COS resource object.

In [24]:
auth_endpoint = 'https://iam.cloud.ibm.com/identity/token'

We create COS resource to be able to write data to Cloud Object Storage.

In [25]:
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

Now you will create bucket in COS and copy `training dataset` for model from **WA_FnUseC_TelcoCustomerChurn.csv**.

In [26]:
from uuid import uuid4

bucket_uid = str(uuid4())

score_filename = "WA_FnUseC_TelcoCustomerChurn.csv"
buckets = ["churn-" + bucket_uid]

In [27]:
for bucket in buckets:
    if not cos.Bucket(bucket) in cos.buckets.all():
        print('Creating bucket "{}"...'.format(bucket))
        try:
            cos.create_bucket(Bucket=bucket)
        except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
            print('Error: {}.'.format(e.response['Error']['Message']))

Creating bucket "churn-2241566d-aa03-4b44-b821-d6cb4989b335"...


In [28]:
bucket_obj = cos.Bucket(buckets[0])

print('Uploading data {}...'.format(score_filename))
with open(filename, 'rb') as f:
    bucket_obj.upload_fileobj(f, score_filename)
print('{} is uploaded.'.format(score_filename))

Uploading data WA_FnUseC_TelcoCustomerChurn.csv...
WA_FnUseC_TelcoCustomerChurn.csv is uploaded.


### 4.1: Save pipeline and model

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [29]:
training_data_references = [
                {
                    "id": "customer churn",
                    "type": "s3",
                    "connection": {
                        "access_key_id": connection_access_key_id,
                        "endpoint_url": service_endpoint,
                        "secret_access_key": connection_secret_access_key
                    },
                    "location": {
                        "bucket": buckets[0],
                        "path": score_filename,
                    }
                }
            ]

In [30]:
saved_model = client.repository.store_model(
    model=model_lr, 
    meta_props={
        client.repository.ModelMetaNames.NAME:'Customer Churn model',
        client.repository.ModelMetaNames.SPACE_UID: space_id,
        client.repository.ModelMetaNames.TYPE: "mllib_2.4",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_id_by_name('spark-mllib_2.4'),
        client.repository.ModelMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
        client.repository.ModelMetaNames.LABEL_FIELD: "Churn",
    },  
    training_data=train_data, 
    pipeline=pipeline_lr)

Get saved model metadata from Watson Machine Learning.

In [31]:
published_model_ID = client.repository.get_model_uid(saved_model)

print("Model Id: " + str(published_model_ID))

Model Id: 1dadccda-6a06-4cc4-865d-79c75538bc3a


**Model Id** can be used to retrive latest model version from Watson Machine Learning instance.

### 4.2: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [32]:
loaded_model = client.repository.load(published_model_ID)

In [33]:
print(type(loaded_model))

<class 'pyspark.ml.pipeline.PipelineModel'>


As you can see the name is correct. You have already learned how save and load the model from Watson Machine Learning repository.

<a id="visualization"></a>
## 5. Predict locally

In this section you will learn how to load data from batch scoring and visualize the prediction results with plotly package.

### 5.1: Make local prediction using previously loaded model and test data

In this subsection you will score *predict_data* data set.

In [34]:
predictions = loaded_model.transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [35]:
predictions.show(5, truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------
 customerID       | 0378-XSZPU                                                                                                                            
 gender           | Male                                                                                                                                  
 SeniorCitizen    | 0                                                                                                                                     
 Partner          | Yes                                                                                                                                   
 Dependents       | No                                                                                                                                    
 tenure           | 58                                                

By tabulating a count, you can see the split between labels.

In [36]:
predictions.select("prediction").groupBy("prediction").count().show(truncate=False)

+----------+-----+
|prediction|count|
+----------+-----+
|0.0       |114  |
|1.0       |34   |
+----------+-----+



<a id="scoring"></a>
## 6. Deploy model and score in a Cloud

In this section you will learn how to create batch deployment and to score a new data record by using the Watson Machine Learning REST API. 
For more information about REST APIs, see the [Swagger Documentation](https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_create).

### 6.1 Prepare scoring data for batch job

#### Get data for prediction

First, download scoring data into notebook's filesystem

In [38]:
import os
from wget import download

sample_dir = 'spark_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename = os.path.join(sample_dir, 'scoreInput.csv')
if not os.path.isfile(filename):
    filename = download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/data/customer_churn/scoreInput.csv", out=sample_dir)

#### Save data for prediction to Object Storage

ibm-cos-sdk library allows Python developers to manage Cloud Object Storage (COS).

In [39]:
import ibm_boto3
from ibm_botocore.client import Config

**Action**: Put credentials from Object Storage Service in Bluemix here.

In [40]:
cos_credentials = {
                  "apikey": "***",
                  "cos_hmac_keys": {
                    "access_key_id": "***",
                    "secret_access_key": "***"
                  },
                  "endpoints": "***",
                  "iam_apikey_description": "***",
                  "iam_apikey_name": "***",
                  "iam_role_crn": "***",
                  "iam_serviceid_crn": "***",
                  "resource_instance_id": "***"
                }

In [41]:
connection_apikey = cos_credentials['apikey']
connection_resource_instance_id = cos_credentials["resource_instance_id"]
connection_access_key_id = cos_credentials['cos_hmac_keys']['access_key_id']
connection_secret_access_key = cos_credentials['cos_hmac_keys']['secret_access_key']

**Action**: Define the service endpoint we will use. <br>
**Tip**: You can find this information in Endpoints section of your Cloud Object Storage intance's dashbord.

In [42]:
service_endpoint = 'https://s3.us.cloud-object-storage.appdomain.cloud'

You also need IBM Cloud authorization endpoint to be able to create COS resource object.

In [43]:
auth_endpoint = 'https://iam.cloud.ibm.com/identity/token'

We create COS resource to be able to write data to Cloud Object Storage.

In [44]:
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

Now you will create bucket in COS and copy `dataset` for batch scoring from **Telco Customer Churn project**.

In [45]:
from uuid import uuid4

bucket_uid = str(uuid4())

score_filename = "scoreInput.csv"
buckets = ["batch-job-" + bucket_uid]

In [46]:
for bucket in buckets:
    if not cos.Bucket(bucket) in cos.buckets.all():
        print('Creating bucket "{}"...'.format(bucket))
        try:
            cos.create_bucket(Bucket=bucket)
        except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
            print('Error: {}.'.format(e.response['Error']['Message']))

Creating bucket "batch-job-58b4e9f2-5c69-4aae-95b7-67bb7430155e"...


In [47]:
bucket_obj = cos.Bucket(buckets[0])

print('Uploading data {}...'.format(score_filename))
with open(filename, 'rb') as f:
    bucket_obj.upload_fileobj(f, score_filename)
print('{} is uploaded.'.format(score_filename))

Uploading data scoreInput.csv...
scoreInput.csv is uploaded.


### 6.2: Create batch deployment

Now you can create a batch scoring endpoint.
Execute the following sample code that uses the **published_model_ID** value to create the scoring endpoint for predictions.

#### Create batch deployment for published model

In [74]:
meta_data = {
    client.deployments.ConfigurationMetaNames.NAME: "Customer Churn batch deployment",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {
        "name": "S-Spark",
        "num_nodes": 1
    }
}

deployment_details = client.deployments.create(published_model_ID, meta_props=meta_data)



#######################################################################################

Synchronous deployment creation for uid: '1dadccda-6a06-4cc4-865d-79c75538bc3a' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='745449c0-f784-4225-b9ce-9a3fb37189ca'
------------------------------------------------------------------------------------------------




Batch deployment has been created.

You can retrieve now your deployment ID.

In [75]:
deployment_uid = client.deployments.get_uid(deployment_details)

You can also list all deployments in your space.

In [ ]:
client.deployments.list()

If you want to get additional information on your deployment, you can do it as below.

In [77]:
client.deployments.get_details(deployment_uid)

{'entity': {'asset': {'id': '1dadccda-6a06-4cc4-865d-79c75538bc3a'},
  'batch': {},
  'custom': {},
  'hardware_spec': {'id': 'd92943ba-9f47-407d-9280-c85281687a1e',
   'name': 'S-Spark',
   'num_nodes': 1},
  'name': 'Customer Churn batch deployment',
  'space_id': '968d9b1a-3cb4-4029-bc57-f53ec5e8743b',
  'status': {'state': 'ready'}},
 'metadata': {'created_at': '2020-08-27T10:43:39.915Z',
  'id': '745449c0-f784-4225-b9ce-9a3fb37189ca',
  'modified_at': '2020-08-27T10:43:39.915Z',
  'name': 'Customer Churn batch deployment',
  'owner': 'IBMid-310002AFR6',
  'space_id': '968d9b1a-3cb4-4029-bc57-f53ec5e8743b'}}

#### Create and run Batch job

In [82]:
job_payload_ref = {
    client.deployments.ScoringMetaNames.INPUT_DATA_REFERENCES: [
        {
            "id": "scoringDataset",
            "type": "s3",
            "connection": {
                "endpoint_url": service_endpoint,
                "access_key_id": connection_access_key_id,
                "secret_access_key": connection_secret_access_key
            },
            "location": {
              "bucket": buckets[0],
                "path": score_filename
            }
        }
    ],
    client.deployments.ScoringMetaNames.OUTPUT_DATA_REFERENCE: {
            "id": "output",
            "type": "s3",
            "connection": {
                "endpoint_url": service_endpoint,
                "access_key_id": connection_access_key_id,
                "secret_access_key": connection_secret_access_key
            },
            "location": {
              "bucket": buckets[0],
                "path": "scoring_output"
            }
    }
}

In [84]:
job = client.deployments.create_job(deployment_uid, meta_props=job_payload_ref)

Now, your job has been submitted to Spark runtime.

You can retrieve now your job ID.

In [97]:
job_id = client.deployments.get_job_uid(job)

You can also list all jobs in your space.

In [ ]:
client.deployments.list_jobs()

If you want to get additional information on your job, you can do it as below.

In [ ]:
client.deployments.get_job_details(job_id)

#### Monitor job execution

Here you can check status of your batch scoring. When status of Spark job is `completed` the results will be written to scoring_output file in Object Storage.

In [ ]:
import time

elapsed_time = 0
while client.deployments.get_job_status(job_id).get('state') != 'completed' and elapsed_time < 300:
    print(f" Current state: {client.deployments.get_job_status(job_id).get('state')}")
    elapsed_time += 10
    time.sleep(10)
if client.deployments.get_job_status(job_id).get('state') == 'completed':
    print(f" Current state: {client.deployments.get_job_status(job_id).get('state')}")
    job_details_do = client.deployments.get_job_details(job_id)
    print(job_details_do)
else:
    print("Job hasn't completed successfully in 5 minutes.")

**Preview scored data**

In this subsection you will load scored data.

Get data with scores from Cloud Object Storage.

In [87]:
bucket_obj.download_file(Key='scoring_output', Filename='result.csv')

'result.csv' file is downloaded to your local file directory

In [88]:
ls res*

result.csv


Loading 'result.csv' file as pandas dataframe

In [92]:
import pandas as pd
results_pdf = pd.read_csv('result.csv', header=None)

Preview the results by calling the *head()* method on the predictions in Pandas data frame.

In [93]:
results_pdf.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,Electronic check,70.7,151.65,Yes,1.000,"(31,[1,3,4,5,6,7,9,11,13,15,17,19,21,23,25,26,...",1.0,"[-0.6714665069908214,0.6714665069908214]","[0.3381685431624339,0.661831456837566]",1.0
1,3638-WEABW,Female,0,Yes,No,58,Yes,Yes,DSL,No,...,Credit card (automatic),59.9,3505.10,No,2.768,"(31,[1,4,5,6,8,10,11,14,15,18,19,21,24,25,29,3...",0.0,"[3.9132571296604866,-3.9132571296604866]","[0.9804158668366412,0.01958413316335884]",0.0
2,8665-UTDHZ,Male,0,Yes,Yes,1,No,No phone service,DSL,No,...,Electronic check,30.2,30.20,Yes,1.000,"(31,[0,5,10,11,14,15,17,19,21,23,26,29,30],[1....",1.0,"[0.05572147387283066,-0.05572147387283066]","[0.5139267652403491,0.48607323475965086]",0.0
3,8773-HHUOZ,Female,0,No,Yes,17,Yes,No,DSL,No,...,Mailed check,64.7,1093.10,Yes,1.000,"(31,[1,3,5,6,7,10,11,13,15,17,20,22,23,25,27,2...",1.0,"[0.646282974444357,-0.646282974444357]","[0.6561723512925712,0.34382764870742877]",0.0
4,4080-IIARD,Female,0,Yes,No,13,Yes,Yes,DSL,Yes,...,Electronic check,76.2,981.45,No,2.768,"(31,[1,4,5,6,8,10,12,14,15,18,20,21,23,25,26,2...",0.0,"[0.5669775770632643,-0.5669775770632643]","[0.6380654747444198,0.3619345252555803]",0.0


You can see the predictions for all the records in pandas data frame.

<a id="cleanup"></a>
## 7. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 8. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our [Online Documentation](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html)
 for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Amadeusz Masny**, Python Software Developer in Watson Machine Learning at IBM

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.